## CESM2 - LARGE ENSEMBLE (LENS2)

#### by Mauricio Rocha and Dr. Gustavo Marques

- The goal of this notebook is to calculate the MOC in density coordinates. We used https://github.com/sgyeager/POP_MOC as reference; thus, We thank Dr. Stephen Yeager. We also thank Michael Levy for the technical support. 

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import scipy.stats
from scipy import signal
from functools import partial
import glob
import dask
import cf_xarray
import intake
import pprint
import intake_esm
import matplotlib.pyplot as plt
from xhistogram.xarray import histogram
import pop_tools
%matplotlib inline
from MOCutils import popmoc
from dask.distributed import Client
from ncar_jobqueue import NCARCluster#,PBSCluster

### Improve the workflow using clusters 

In [ ]:
mem_per_worker = 90 # in GB more memory here maybe 100 GB
num_workers = 45 # more workers maybe 45
cluster = NCARCluster(cores=1, processes=1, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=1:mem={mem_per_worker}GB')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

### Read in OGCM history file & MOC template file

In [ ]:
fmoc = '/glade/u/home/yeager/analysis/python/POP_MOC/moc_template.nc'
ds_moctemp = xr.open_dataset(fmoc) # MOC template

In [ ]:
# Open original collection description file
cat_url='/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
col = intake.open_esm_datastore(cat_url)

In [ ]:
# Catolog
print("Catalog file:", col.esmcol_data["catalog_file"])
col.df.head(10)

In [ ]:
uniques_orig = col.unique(columns=["component", "frequency", "experiment", "variable"])
pprint.pprint(uniques_orig, compact=True, indent=1, width=80)

In [ ]:
# Some variables like temperature, salinity, are not available for annual frequency, so we chose the monthly frequency.
col.search(component="ocn", variable=["TEMP","SALT","UVEL","VVEL"], frequency="month_1", experiment="historical").df

In [ ]:
%%time
cat_subset = col.search(component='ocn', # ocean component
                            variable=['TEMP','SALT','UVEL','VVEL'], # temperature, salinity, zonal velocity, meridional velocity
                            frequency='month_1', # monthly
                            experiment='historical', # 1850-2014
                            forcing_variant='smbb', # you can use smbb or cmip6
                       )
dset_dict_raw = cat_subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
print(f"\nDataset dictionary keys:\n {dset_dict_raw.keys()}")

### Compute sigma-2 field from LENS2 dataset

In [ ]:
tslice = slice("1960-01-01", "2014-12-31") # select the period you wish

#### Salinity

In [ ]:
%%time
ds_smbb_salt = dset_dict_raw['ocn.historical.pop.h.smbb.SALT'] # Salinity
print(f"Salinty before: {dask.utils.format_bytes(ds_smbb_salt.nbytes)}")
ds_smbb_salt = ds_smbb_salt.sel(time=tslice)
ds_smbb_salt = ds_smbb_salt.resample(time='1Y', closed='left').mean('time') # Yearly average
salt = ds_smbb_salt['SALT']
salt = salt.mean(dim = ["member_id"]) # Average of all members
print(f"Salinty after: {dask.utils.format_bytes(salt.nbytes)}")
salt = salt.load() # Necessary because pop-tools.eos() doesn't play nicely with dask
del ds_smbb_salt

#### Temperature

In [ ]:
%%time
ds_smbb_temp = dset_dict_raw['ocn.historical.pop.h.smbb.TEMP'] # Temperature
print(f"Temperature before: {dask.utils.format_bytes(ds_smbb_temp.nbytes)}")
ds_smbb_temp = ds_smbb_temp.sel(time=tslice)
ds_smbb_temp = ds_smbb_temp.resample(time='1Y', closed='left').mean('time') # Yearly average
temp = ds_smbb_temp['TEMP']
temp = temp.mean(dim = ["member_id"]) # Average of all members
print(f"Temperature after: {dask.utils.format_bytes(temp.nbytes)}")
temp = temp.load() # Necessary because pop-tools.eos() doesn't play nicely with dask

#### Zonal velocity

In [ ]:
%%time
ds_smbb_uvel = dset_dict_raw['ocn.historical.pop.h.smbb.UVEL'] #  Zonal velocity 
print(f"Zonal Velocity before: {dask.utils.format_bytes(ds_smbb_uvel.nbytes)}")
ds_smbb_uvel = ds_smbb_uvel.sel(time=tslice)
ds_smbb_uvel = ds_smbb_uvel.resample(time='1Y', closed='left').mean('time') # Yearly average
uvel = ds_smbb_uvel['UVEL']
uvel = uvel.mean(dim = ["member_id"]) # Average of all members
print(f"Zonal Velocity before: {dask.utils.format_bytes(uvel.nbytes)}")
uvel = uvel.load() # Necessary because pop-tools.eos() doesn't play nicely with dask

#### Meridional Velocity

In [ ]:
%%time
# Meridional velocity 
ds_smbb_vvel = dset_dict_raw['ocn.historical.pop.h.smbb.VVEL'] #  meridional velocity 
print(f"Meridional Velocity before: {dask.utils.format_bytes(ds_smbb_vvel.nbytes)}")
ds_smbb_vvel = ds_smbb_vvel.sel(time=tslice)
ds_smbb_vvel = ds_smbb_vvel.resample(time='1Y', closed='left').mean('time') # Yearly average
vvel = ds_smbb_vvel['VVEL']
vvel = vvel.mean(dim = ["member_id"]) # Average of all members
print(f"Zonal Velocity before: {dask.utils.format_bytes(vvel.nbytes)}")
vvel = vvel.load() # Necessary because pop-tools.eos() doesn't play nicely with dask

#### Define k-index array

In [ ]:
dims = np.shape(temp)
#ne = dims[0] # ensember member
nt = dims[0]  # time 
nz = dims[1]  # depth
ny = dims[2]  # latitude
nx = dims[3]  # longitude
kji = np.indices((nz,ny,nx))
kindices = kji[0,:,:,:] + 1 

#### Define sigma2_T

In [ ]:
refz = 2000 # reference depth
refdep = xr.full_like(salt,refz).rename('REFDEP')
# Sigma2 on model TLAT, TLONG
sigma2_T = pop_tools.eos(salt=salt,temp=temp,depth=refdep) - 1000
sigma2_T = sigma2_T.assign_attrs({'long_name':'Sigma referenced to {}m'.format(refz),'units':'kg/m^3'})
sigma2_T = sigma2_T.mean(dim=["time"]) # Average over time
# apply T-grid mask
#mask=kindices<=ds['KMT'].values[None,:,:]
#sigma2_T = sigma2_T.where(mask)

### Define target sigma-2 vertical grid

In [ ]:
# Use predefined 86-layer sigma2 grid:
sigma_mid,sigma_edge = popmoc.sigma2_grid_86L()

### Compute MOC(Sigma2) using xhistogram 

#### 1. Compute Isopycnal Layer Thickness

In [ ]:
# Here, test histogram by counting cells in each density bin. Vertical sum should be same as KMT.
iso_count = histogram(sigma2_T, bins=[sigma_edge.values],dim=['z_t'],density=False)
iso_count = iso_count.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
kmtdiff = (iso_count.sum('sigma') - ds_smbb_temp['KMT'].mean(dim=["time"]))
print("Max difference from true KMT = {}".format(abs(kmtdiff).max().values))

In [ ]:
# Use histogram to compute layer thickness. Vertical sum should be same as HT.
dzwgts = (ds_smbb_temp['dz']/100.).assign_attrs({'units':'m'})
dzwgts = dzwgts.mean(dim=["time"]) # Average over time
iso_thick = histogram(sigma2_T, bins=[sigma_edge.values], weights=dzwgts,dim=['z_t'],density=False)
iso_thick = iso_thick.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
iso_thick = iso_thick.rename('Isopycnal Layer Thickness').assign_attrs({'units':'m'})
htdiff = iso_thick.sum('sigma') - (ds_smbb_temp['HT']/100.).assign_attrs({'units':'m'})
htdiff = htdiff.mean(dim=["time"]) # Average over time
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))
#In the original Notebook, the maximum difference is: HT = 1.2270752449694555e-05m

#### 2. Compute Isopycnal Layer Depth

In [ ]:
# Cumulative sum of layer thickness yields depth of layer edges:
iso_depth = iso_thick.cumsum('sigma').rename('Isopycnal Layer Depth')
iso_depth['sigma'] = sigma_edge.isel(sigma=slice(1,None))

In [ ]:
iso_depth.isel(sigma=84).plot(size=6,vmax=5500)

In [ ]:
# Isopycnal depth of bottom edge should be same as HT.
htdiff =  iso_depth.isel(sigma=-1) - (ds_smbb_temp['HT']/100.).assign_attrs({'units':'m'})
htdiff = htdiff.mean(dim=["time"]) # Average over time
print("Max difference from true HT = {}m".format(abs(htdiff).max().values))
#Max difference from true HT = 1.2270752449694555e-05m

#### 3. Compute Isopycnal Layer Horizontal Volume Flux

In [ ]:
# Grid-oriented Volume FLuxes:
uvel = uvel.where(uvel<1.e30).fillna(0.)
vvel = vvel.where(vvel<1.e30).fillna(0.)
uvel = (uvel*ds_smbb_uvel['DYU']*ds_smbb_uvel['dz']/1.e6).assign_attrs({'units':'m^3/s'})
vvel = (vvel*ds_smbb_vvel['DXU']*ds_smbb_vvel['dz']/1.e6).assign_attrs({'units':'m^3/s'})
uvel = uvel.mean(dim=["time"]) # Average over time
vvel = vvel.mean(dim=["time"]) # Average over time

In [ ]:
# Volume fluxes in density-space. Vertical sum is density-space should reproduce vertical sum in depth-space.
iso_uflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=uvel,dim=['z_t'],density=False) # The 'numpy.histogram_bin_edges' function is not implemented by Dask array.
iso_uflux = iso_uflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})
iso_vflux = histogram(sigma2_T, bins=[sigma_edge.values],weights=vvel,dim=['z_t'],density=False)
iso_vflux = iso_vflux.rename({'density_bin':'sigma'}).assign_coords({'sigma':sigma_mid})

ufluxdiff = iso_uflux.sum('sigma') - uvel.sum('z_t')
vfluxdiff = iso_vflux.sum('sigma') - vvel.sum('z_t')
print("Max difference from true Uflux = {}".format(abs(ufluxdiff).max().values))
print("Max difference from true Vflux = {}".format(abs(vfluxdiff).max().values))
#Max difference from true Uflux = 1367813.3765927013
#Max difference from true Vflux = 456888.7641561439

Need to investigate these differences, which appear to be associated with overflows. The difference plot below shows zero almost everywhere except near Nordic Seas overflow points.

In [ ]:
ufluxdiff.plot(size=7,vmin=-1.e5,vmax=1.e5)

#### 4. Compute Vertical Volume Flux using model divergence operator

In [ ]:
wflux = popmoc.pop_isowflux(iso_uflux,iso_vflux,'sigma',sigma_edge)

#### 5. Compute Zonal Sums of Vertical Volume Flux in latitude strips

In [ ]:
# Load predefined 1-degree target latitude grid:
lat_mid,lat_edge = popmoc.latitude_grid_1deg()

In [ ]:
## Define MOC region mask with legend:
rmask = ds_smbb_temp.REGION_MASK
rmask=rmask.mean(dim=["time"])
rmaskmoc = rmask.where(rmask>0)
rmaskmoc = xr.where((rmask>0),1,rmaskmoc)
rmaskmoc = xr.where((rmask>=6) & (rmask<=11),2,rmaskmoc)
rmaskmoc.plot(levels=[0,1,2,3]);
rmaskmoc.attrs['legend'] = {0:"Global",1:"IndoPac+SO",2:"Atlantic"}

In [ ]:
tarea = ds_smbb_temp['TAREA']
tarea=tarea.mean(dim=["time"])
tlat = ds_smbb_temp['TLAT']
wflux_zonsum = popmoc.mesh_zonalavg(wflux,tarea,tlat,rmaskmoc,rmaskmoc.legend,lat_edge,sum=True)

#### 6. Compute cumulative meridional integral of zonally-summed wflux

A southward cumulative integral from 90N avoids issues associated with southern boundary of Atlantic region.

In [ ]:
moc = -wflux_zonsum.sel(lat=slice(None,None,-1)).cumsum('lat').sel(lat=slice(None,None,-1))
moc = (moc/1.e6).assign_attrs({'units':'Sv'})   
moc.name = 'MOC'

In [ ]:
moc.isel(region=0).plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])

In [ ]:
moc.isel(region=1).plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])

In [ ]:
moc.isel(region=2).plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])

In [ ]:
moc.isel(region=[1,2]).sum('region').plot(size=7,vmax=40,levels=21)
plt.ylim([38,29])